Automated daily NHL games and odds informations

1. Import the required libraries

In [1]:
import requests
import pandas as pd
import time
from datetime import date, datetime, timedelta
from dateutil import parser

2. Get today's NHL games of the day

In [2]:
todayDate = date.today().strftime("%Y-%m-%d")
statsToday = "https://statsapi.web.nhl.com/api/v1/schedule?date=" + todayDate
requestToday = requests.get(statsToday).json()

#Build the dataframe
gameToday = pd.DataFrame(columns=["HomeTeam", "AwayTeam"])

for teams in requestToday["dates"][0]["games"]:
    away = teams["teams"]["away"]["team"]["name"]
    home = teams["teams"]["home"]["team"]["name"]
    tempgamesToday = pd.DataFrame([[home, away]], columns=['HomeTeam', 'AwayTeam'])
    gamesToday = pd.concat([gameToday,tempgamesToday], ignore_index=True)

gamesToday["Teams"] = gamesToday["HomeTeam"] + " - " + gamesToday["AwayTeam"]
gamesToday

,HomeTeam,AwayTeam,Teams
0,Winnipeg Jets,St. Louis Blues,Winnipeg Jets - St. Louis Blues


3. Game statistics from a specific date

In [3]:
startDate = "2019-01-01"
endDate = date.today() + timedelta(days=-1)
endDate = endDate.strftime("%Y-%m-%d")
statsGame = f"https://statsapi.web.nhl.com/api/v1/schedule?startDate={startDate}&endDate={endDate}"
requestGame = requests.get(statsGame).json()

#Build the dataframe
gameStats = pd.DataFrame(columns=["HomeTeam", "AwayTeam", "Goals"])

for dates in requestGame["dates"]:
    for stats in dates["games"]:
        away = stats["teams"]["away"]["team"]["name"]
        home = stats["teams"]["home"]["team"]["name"]
        awayGoals = stats["teams"]["away"]["score"]
        homeGoals = stats["teams"]["home"]["score"]
        goals = awayGoals + homeGoals
        gameStats1 = pd.DataFrame([[home, away, goals]], columns=['HomeTeam', 'AwayTeam', 'Goals'])
        gameStats = pd.concat([gameStats,gameStats1], ignore_index=True)

gameStats

,HomeTeam,AwayTeam,Goals
0,Chicago Blackhawks,Boston Bruins,6
1,Nashville Predators,Philadelphia Flyers,4
2,Vegas Golden Knights,Los Angeles Kings,2
3,Ottawa Senators,Vancouver Canucks,7
4,Detroit Red Wings,Calgary Flames,8
...,...,...,...
5443,Seattle Kraken,Columbus Blue Jackets,4
5444,Edmonton Oilers,Chicago Blackhawks,10
5445,Anaheim Ducks,Arizona Coyotes,3
5446,Carolina Hurricanes,Boston Bruins,5


4. The statistic is based on bets under/over 5.5 goals, so the variable is defined.

In [11]:
booleanNumber = 6

gameStats.loc[gameStats["Goals"] >= booleanNumber, "O/U Percentage"] = 1
gameStats.loc[gameStats["Goals"] < booleanNumber, "O/U Percentage"] = 0
gameBoolean = gameStats.groupby(["HomeTeam", "AwayTeam"]).mean().reset_index()
gameBoolean["Teams"] = gameBoolean["HomeTeam"] + " - " + gameBoolean["AwayTeam"]

gamesData = gamesToday.merge(gameBoolean, how="inner", on="Teams")
gamesData = gamesData[["Teams", "O/U Percentage"]]
gamesData["O/U Percentage"] = gamesData["O/U Percentage"].apply(lambda x: "{:.0f}".format(x * 100) + "%")
gamesData

,Teams,O/U Percentage
0,Winnipeg Jets - St. Louis Blues,56%


5. Collect the daily Odds

In [13]:
api_key = '778f74cbbc921917acd532fa7b44aa37'
sport = 'icehockey_nhl'
region = 'eu'
market = 'totals'
urlOdds = f"https://api.the-odds-api.com//v4/sports/{sport}/odds/?apiKey={api_key}&regions={region}&markets={market}"
requestOdds = requests.get(urlOdds).json()

6. Definition of betting type and get the daily odds

In [16]:
betType = 5.5

#Build the DataFrame
oddsToday = pd.DataFrame(columns=["HomeTeam", "AwayTeam", "BetCompany", "OddsOver", "OddsUnder"])

for games in requestOdds:
    datetime_string = games["commence_time"]
    datetime_object = parser.parse(datetime_string).date()
    dateTomorrow = datetime.today().date() + timedelta(days=1)
    if datetime_object == dateTomorrow:
        home = games["home_team"]
        away = games["away_team"]
        for office in games["bookmakers"]:
            if office["markets"][0]["outcomes"][0]["point"] == betType:
                over = office["markets"][0]["outcomes"][0]["price"]
                under = office["markets"][0]["outcomes"][1]["price"]
                company = office["title"]
                tempoddsToday = pd.DataFrame([[home, away, company, over, under]], columns=['HomeTeam', 'AwayTeam', "BetCompany", 'OddsOver', 'OddsUnder'])
                oddsToday = pd.concat([oddsToday,tempoddsToday], ignore_index=True)

oddsToday["Teams"] = oddsToday["HomeTeam"] + " - " + oddsToday["AwayTeam"]
oddsToday

,HomeTeam,AwayTeam,BetCompany,OddsOver,OddsUnder,Teams
0,Winnipeg Jets,St Louis Blues,888sport,1.70,2.20,Winnipeg Jets - St Louis Blues
1,Winnipeg Jets,St Louis Blues,1xBet,1.76,2.23,Winnipeg Jets - St Louis Blues
2,Winnipeg Jets,St Louis Blues,Matchbook,1.78,2.16,Winnipeg Jets - St Louis Blues


7. Merge the DataFrames amd export to .xlsx format

In [22]:
DataFrame = oddsToday.merge(gamesData, how='inner', on="Teams")
DataFrame.to_excel("4_NHL_Analysis.xlsx", index=False)
DataFrame

,HomeTeam,AwayTeam,BetCompany,OddsOver,OddsUnder,Teams,O/U Percentage
